<a href="https://colab.research.google.com/github/kishore145/AI-ML-Foundations/blob/master/Neural%20Networks/20_Transfer_Learning_demo_using_VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Load dependencies
from tensorflow.keras.applications.vgg19 import VGG19 # Import VGG19
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [3]:
# Load pretrained VGG19 model
vgg19 = VGG19(weights='imagenet',
              include_top=False,
              input_shape = (224,224,3), 
              pooling = None)

80142336/80134624 [==============================] - 1s 0us/step


In [4]:
# Freeze all layers in VGG19 to preserve learning from imagenet
for layer in vgg19.layers:
  layer.trainable = False
  

In [5]:
# Adding custom classification layers
# Instantiate a sequential model with vgg19 layers
model = Sequential()
model.add(vgg19)

# Add custom classification layer on top
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

In [6]:
model.summary()
# As seen on the below data, the number of trainable parameters is only 50k
# This shows that we are utilizing the pre-trained weights for detecting low  
# level features of the image and only training higher abstract layer to differentiate
# hot dog and not hot dog.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 50178     
Total params: 20,074,562
Trainable params: 50,178
Non-trainable params: 20,024,384
_________________________________________________________________


In [7]:
model.compile(optimizer='nadam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
# Data preparation for model training
# You can comment out the two lines of code below if you executed them on
# a previous run of the notebook. The wget command downloads the data and the 
# tar command extracts the data from a compressed file format. 
! wget -c https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
! tar -xzf hot-dog-not-hot-dog.tar.gz

--2020-06-29 06:03:01--  https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz [following]
--2020-06-29 06:03:01--  https://www.dropbox.com/s/raw/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb0c4768d62b64c7a7555eb7370.dl.dropboxusercontent.com/cd/0/inline/A6htMTXUd2b5yHrsevxAEA8zTkoNApu9KKiY0uKVVS2GPiB2qXbYp1S12TWO_h6jAm6BLW4J0k9dFvGmgo8bwnbc4WnYhAvTMIfHV-dI2iXd7XejqbOuCO7kitWYa12FbB8/file# [following]
--2020-06-29 06:03:02--  https://ucb0c4768d62b64c7a7555eb7370.dl.dropboxusercontent.com/cd/0/inline/A6htMTXUd2b5yHrsevxAEA8zTkoNApu9KKiY0uKVVS2GPiB2qXbYp1S12TWO_h6jAm6BLW4J

In [9]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255, 
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [10]:
# Define the batch size:
batch_size=32

In [11]:
# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train', 
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [12]:
# Fit pre-trained model
model.fit(train_generator, steps_per_epoch=15, 
                    epochs=16, validation_data=valid_generator, 
                    validation_steps=15)

Epoch 1/16
15/15 [==============================] - 575s 38s/step - loss: 1.1069 - accuracy: 0.5579 - val_loss: 0.5314 - val_accuracy: 0.7250
Epoch 2/16
15/15 [==============================] - 581s 39s/step - loss: 1.0284 - accuracy: 0.6083 - val_loss: 0.4957 - val_accuracy: 0.7646
Epoch 3/16
15/15 [==============================] - 573s 38s/step - loss: 0.7846 - accuracy: 0.6781 - val_loss: 0.4944 - val_accuracy: 0.7812
Epoch 4/16
15/15 [==============================] - 582s 39s/step - loss: 0.7029 - accuracy: 0.6931 - val_loss: 0.5706 - val_accuracy: 0.7417
Epoch 5/16
15/15 [==============================] - 574s 38s/step - loss: 0.5648 - accuracy: 0.7597 - val_loss: 1.4239 - val_accuracy: 0.5625
Epoch 6/16
15/15 [==============================] - 577s 38s/step - loss: 0.7187 - accuracy: 0.7189 - val_loss: 0.6226 - val_accuracy: 0.7333
Epoch 7/16
15/15 [==============================] - 573s 38s/step - loss: 0.5862 - accuracy: 0.7747 - val_loss: 0.6268 - val_accuracy: 0.7333
Epoch 

In [ ]:
# With transfer learning, we can get reasonable accuracy with as little as 16 epochs
# and can train it on CPU for experimentation. 